# use estimator 
<b> estimator을 기본으로 하는 간단한 logistic regression을 만든다.  
<b> source 설명 이후 실습을 진행한다.  

In [0]:
sample = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

label =[[1], [0], [1], [1], [0], [1]]

In [0]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras import preprocessing

In [0]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sample) 
sequence = tokenizer.texts_to_sequences(sample) 

word_index = tokenizer.word_index

In [0]:
EPOCH_SIZE = 100
BATCH_SIZE = 1

def train_input_fn():
  dataset = tf.data.Dataset.from_tensor_slices((sequence, label))
  dataset = dataset.repeat(EPOCH_SIZE)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.shuffle(len(sequence))
  iterator = dataset.make_one_shot_iterator()
  
  return iterator.get_next()

In [0]:
VOCAB_SIZE = len(word_index) + 1
EMB_SIZE = 128

def model_fn(features, labels, mode):
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    
    embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
    embed_input = tf.reduce_mean(embed_input, axis=-1)
    
    hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    output = tf.nn.sigmoid(output_layer)
    
    loss = tf.losses.mean_squared_error(output, labels)

    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(
                  mode=mode,
                  train_op=train_op,
                  loss=loss)

In [7]:
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6aa1388d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
estimator.train(train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./data_out/checkpoint/dnn/model.ckpt-600
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 600 into ./data_out/checkpoint/dnn/model.ckpt.
INFO:tensorflow:loss = 0.0019326176, step = 601
INFO:tensorflow:global_step/sec: 697.277
INFO:tensorflow:loss = 0.00014069525, step = 701 (0.148 sec)
INFO:tensorflow:global_step/sec: 1083.73
INFO:tensorflow:loss = 0.00028271438, step = 801 (0.092 sec)
INFO:tensorflow:global_step/sec: 1135.16
INFO:tensorflow:loss = 0.00012013951, step = 901 (0.089 sec)
INFO:tensorflow:global_step/sec: 1128.42
INFO:tensorflow:loss = 9.120089e-05, step 